# Finding a volume's Extracted Features data from a HathiTrust volume ID

In [ ]:
from htrc_features import utils
utils.id_to_rsync('miun.adx6300.0001.001')

'miun/pairtree_root/ad/x6/30/0,/00/01/,0/01/adx6300,0001,001/miun.adx6300,0001,001.json.bz2'

When you install the HTRC Feature Reader, a command line utility is installed, `htid2rsync`, which does this same conversion process.

```bash
$ htid2rsync --help
usage: htid2rsync [-h] id [id ...]

Convert a HathiTrust ID to a pairtree path for Rsyncing that id's Extracted
Features dataset file. This does not check if the file exists.

positional arguments:
  id          A HathiTrust id or multiple ids to convert.
```

As is noted in the help message above, it can be used with a single ID or multiple IDs.

```bash
$ htid2rsync miun.adx6300.0001.001 hvd.32044010273894
miun/pairtree_root/ad/x6/30/0,/00/01/,0/01/adx6300,0001,001/miun.adx6300,0001,001.json.bz2
hvd/pairtree_root/32/04/40/10/27/38/94/32044010273894/hvd.32044010273894.json.bz2
```

This should work on all operating systems. If not, leave a bug report at https://github.com/htrc/htrc-feature-reader.

On Unix or Linux command lines, you can send these files directly into RSync by specifying `--files-from=-`, which tells Rsync to listen for a file list sent in from the previous command with a pipe (`|`):

```bash
$ htid2rsync miun.adx6300.0001.001 hvd.32044010273894 | rsync -azv --files-from=- data.sharc.hathitrust.org::pd-features/merged/ my-local-folder/
```

### Without the HTRC Feature Reader Library

The filepath to sync Extracted Features files through RSync follows a [pairtree format](https://wiki.ucop.edu/display/Curation/PairTree?preview=/14254128/16973838/PairtreeSpec.pdf).

Because the official pairtree library is only compatible with Python 2.X, we recommend just using the following functions for encoding a volume id to a filename-friendly format and converting the safe id to a file path: 

In [ ]:
def id_encode(id):
    return id.replace(":", "+").replace("/", "=").replace(".", ",")

def id2path(id):
    clean_id = id_encode(id)
    path = []
    while len(clean_id) > 0:
        val, clean_id = clean_id[:2], clean_id[2:]
        path.append(val)
    return '/'.join(path)

This method brings it all together, generating the pairtree path which can then be downloaded through RSync:

In [ ]:
def id_to_rsync(htid):
    '''
    Take an HTRC id and convert it to an Rsync location for syncing Extracted
    Features
    '''
    libid, volid = htid.split('.', 1)
    volid_clean = id_encode(volid)
    filename = '.'.join([libid, volid_clean, 'json.bz2'])
    path = '/'.join([libid, 'pairtree_root', id2path(volid).replace('\\', '/'),
                     volid_clean, filename])
    return path

For example,

In [ ]:
id_to_rsync('miun.adx6300.0001.001')

'miun/pairtree_root/ad/x6/30/0,/00/01/,0/01/adx6300,0001,001/miun.adx6300,0001,001.json.bz2'

The Extracted Features for this volume can be downloaded using RSync:

```
rsync -azv data.sharc.hathitrust.org::pd-features/{{URL}} .
```

## Compiling and downloading a list of volumes

`select.txt` contains a set of ids for 10k HathiTrust Digital Library volumes in the PZ class (_Fiction and juvenile belles lettres_), that were collected from the HTRC from though its Solr Proxy:

http://chinkapin.pti.indiana.edu:9994/solr/meta/select/?q=callnumber:PZ*&wt=csv&fl=id&rows=10000

Here is what the id's look like:

In [ ]:
idlist = open("select.txt", "r+").readlines()
idlist = [id.strip() for id in idlist[1:]]
print(idlist[:3])

['mdp.39015030727963', 'uc2.ark:/13960/t5cc0v81s', 'miun.adx6300.0001.001']


In [ ]:
rsynclist = [id_to_rsync(v) for v in idlist]
rsynclist[:2]

['mdp/pairtree_root/39/01/50/30/72/79/63/39015030727963/mdp.39015030727963.json.bz2',
 'uc2/pairtree_root/ar/k+/=1/39/60/=t/5c/c0/v8/1s/ark+=13960=t5cc0v81s/uc2.ark+=13960=t5cc0v81s.json.bz2']

We can also write the full list of our desired volume urls to a file and tell rsync to download from that list.

In [ ]:
# Write to file
rsyncf = open('rsync-urls.txt', 'w+')
rsyncf.write("\n".join(rsynclist))
rsyncf.close()

Syncing from a file of URLs can be done as follows:

```
rsync -azv --files-from=rsync-urls.txt data.sharc.hathitrust.org::pd-features/ files/
```

If you don't need the full pairtree directory structure, it can be flattened to a single folder. This example uses [GNU Parallel](http://www.gnu.org/software/parallel/), available for Linux or Mac OS, or installed on Cygwin in Windows.

```
find sample-files/ -type -f | parallel --eta mv {} just-one-folder/
```

## Explanation of ID-to-URL encoding

In [ ]:
htid = 'miun.adx6300.0001.001'
libid, volid = htid.split('.', 1)
print("Institution:\t%s\nId:\t\t%s" % (libid, volid))

Institution:	miun
Id:		adx6300.0001.001


The HathiTrust ID has a library id and an identifier within that library. In this case, _miun_ notes the origin of the file as Michigan University.

In [ ]:
volid_clean = id_encode(volid)
filename = ".".join([libid, volid_clean, 'json.bz2'])
print("Filename:\t%s" % filename)

Filename:	miun.adx6300,0001,001.json.bz2


Some IDs don't play nice with filesystems and need to be encoded in a cleaner format: ":" becomes "+", "/" becomes "=", and "." becomes "," (as above). The institution identifier and the encoded version of the institution's local id become the filename.

In [ ]:
pairtree_root = [libid, 'pairtree_root']
path = '/'.join(pairtree_root + [id2path(volid).replace('\\', '/'),
                                 volid_clean, filename])
print("Pairtree Root:\t%s" % pairtree_root)
print("Full Path:\t%s" % path)

Pairtree Root:	['miun', 'pairtree_root']
Full Path:	miun/pairtree_root/ad/x6/30/0,/00/01/,0/01/adx6300,0001,001/miun.adx6300,0001,001.json.bz2


The institutional id is split up and encoded, then recombined into a path.